## Map Visualisation with Folium and Ipywidgets
### Choropleth of Hong Kong 2016 By-census Results
By-census results are undoubtedly important for various purposes including policy making, business and research proposes, etc. To gain insight from the data, visualisation is crucial. A choropleth can be particularly useful when dealing with geospatial data. This notebook uses the pre-processed data of Hong Kong 2016 By-census results in another notebook "data_preprocessing" to generate interactive choropleth using Folium and Ipywidgets.

In [1]:
import folium
import json
import os
import pandas as pd
import geopandas as gpd
from IPython.core.display import display, HTML
from IPython.display import IFrame
import ipywidgets as widgets
%matplotlib inline
display(HTML("<style>.container { width:80% !important; }</style>")) # widen the cell to display map

In [2]:
# Make functions to facilitate the generation of choropleth, tooltip. 

def make_choropleth(data,name,col,**kargs):
    '''create choropleth from the passed in data'''
    choropleth = folium.Choropleth(
                    name = name,
                    geo_data = geo_json_data,
                    data = data,
                    columns = ["name", col],
                    key_on = "feature.properties.name",
                    fill_opacity = 0.5,
                    line_weight = 1.5,
                    highlight = True,
                    show = False,**kargs)
    return choropleth

def make_tooltip(display,style,col=None):
    '''create tooltip for district name and additional column if col is not None'''
    if col is not None:
        tooltip = folium.features.GeoJsonTooltip(["name",col],
                                             display,
                                             labels=True)
        return  folium.GeoJson(data[["name","geometry",col]],
                                tooltip=tooltip,
                                style_function = lambda x: style)
    else:
        tooltip = folium.features.GeoJsonTooltip(["name"],display,labels=False)
        return folium.GeoJson(data[["name","geometry"]],tooltip=tooltip,
                              style_function = lambda x: style)

def make_map(*args,**kargs):
    '''combine make_choropleth and make_tooltip to facilitate the map creation'''
    m = folium.Map(location=[22.3742, 114.1685],zoom_start=11,min_zoom=11,max_zoom=16)
    choropleth = make_choropleth(*args,**kargs)
    tooltip = make_tooltip(["District"],style,col=None)
    return m.add_child(choropleth.add_child(tooltip))

def show(ethnicity):
    '''used as the passed in function for ipywidgets.interact'''
    return IFrame(ethnicity,1100,800)


Import the geography data and by-census data and combine them into a dataframe for convenience. The geospatial data is in geojson format thus using geopandas and json to read the data can be more useful. The demographic_summary_2016.csv is generated from the preprocessing notebook which provide useful statistic like median age and the ratio of ethnicity in a district.

In [3]:
# Path of map data
geo_path = os.path.join("data","districts.geojson") 
geo_json_data = json.load(open(geo_path))
gpf = gpd.read_file(geo_path)

# Path of statistic data
stat_path =os.path.join("data","demographic_summary_2016.csv") 
district = pd.read_csv(stat_path)

# merge the data into a single DataFrame
merge = gpf.merge(district,"inner",left_on=gpf.name,right_on=district.iloc[:,0])
data = merge.drop(["key_0","Unnamed: 0"],axis=1)

Folium has a Choropleth class which has already provided various useful settings for creating a choropleth. This choropleth contains two layers, one is the median age of male in every district, another is the median age of female in every district. In the right-hand side layer control panel, user can choose the layer they want. The name of the distrcit will be display as the tooltip when the cursor is within its boundary. It is also worth to point out that when the median age layer is displaying, the tooltip will also include the median age of that district. Zooming is also provided although the zooming level is constrained.

In [4]:
m = folium.Map(location=[22.3742, 114.1685],zoom_start=10,min_zoom=10,max_zoom=16)
# Display and style for tooltip
display = ["District","Median Age"]
style = {'fillColor': '#00000000','color': '#00000000'}
base_style = {'color':'#0C090A','fillColor':'#00000000'}

# Create the base tooltip to display the boundary and name of districts
base_tooltip = make_tooltip(["District"],base_style,col=None)
base_tooltip.control = False
base_tooltip.add_to(m)

# Parameters for age choropleth (Name, column, Legend name)
ma_male = ("Median Age Male","median_Male","Median Age of Male")
ma_female = ("Median Age Female","median_Female","Median Age of Female")
median_age = [ma_male, ma_female] # Combining them into a list
median_age_data = data[["name","median_Male","median_Female"]]
# Create choropleth of median age
for each in median_age:
    tooltip = make_tooltip(display,style,col=each[1])
    choropleth = make_choropleth(median_age_data,each[0],each[1],fill_color="PuRd",legend_name=each[2])
    choropleth.add_child(tooltip)
    choropleth.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m # show the map in the cell

However, when there are more layers the display, especially the legends, can be messy. Ipywidgets.interact allows us to select the choropleth the IFrame to display based on our need. 

In [5]:
# Parameters for ethnicity choropleth
ethnicity = ["Filipino","Indonesian","White","Indian","Nepalese","Pakistani","Thai","Japanese"]
ratio = ["Filipino_Total_r","Indonesian_Total_r","White_Total_r",
        "Indian_Total_r","Nepalese_Total_r","Pakistani_Total_r","Thai_Total_r","Japanese_Total_r"]
legend = ["% of {0}".format(each) for each in ethnicity]
combined_set = zip(ethnicity,ratio,legend)

# Create choropleth of ethnicity    
for each in combined_set:
    col_percentage = "{0} in %".format(each[1])
    data[col_percentage] = data[each[1]]*100
    choropleth = make_map(data[["name",col_percentage]],each[0],col_percentage,fill_color="YlGnBu",legend_name=each[2])
    #choropleth.save(os.path.join("map","{0}".format(each[1])+".html")) # uncomment to save the choropleth 

In [6]:
eth_map = []
for each in ratio:
    eth_map.append(os.path.join("map","{0}.html".format(each)))
menu = zip(ethnicity,eth_map)
widgets.interact(show,ethnicity=[each for each in menu])

interactive(children=(Dropdown(description='ethnicity', options=(('Filipino', 'map/Filipino_Total_r.html'), ('…

<function __main__.show(ethnicity)>